In [1]:
import pandas as pd

In [2]:
# data path variable
data_path = "../data/raw/"

orders = pd.read_csv(f"{data_path}olist_orders_dataset.csv")
orderitems = pd.read_csv(f"{data_path}olist_order_items_dataset.csv")
orderpayments = pd.read_csv(f"{data_path}olist_order_payments_dataset.csv")
orderreviews = pd.read_csv(f"{data_path}olist_order_reviews_dataset.csv")
customers = pd.read_csv(f"{data_path}olist_customers_dataset.csv")
sellers = pd.read_csv(f"{data_path}olist_sellers_dataset.csv")
geolocation = pd.read_csv(f"{data_path}olist_geolocation_dataset.csv")
products = pd.read_csv(f"{data_path}olist_products_dataset.csv")
product_category_name_translation = pd.read_csv(f"{data_path}product_category_name_translation.csv")

# translate product category names to English
products = products.merge(product_category_name_translation, on='product_category_name', how='left').drop('product_category_name', axis=1) \
    .rename(columns={'product_category_name_english': 'product_category_name'})

In [3]:
# eda for orders
# status and date columns
date_cols = ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']
for c in date_cols:
    orders[c] = pd.to_datetime(orders[c], errors='coerce')

# compute percentage and count (transpose so rows = date columns)
have_pct = orders.groupby('order_status')[date_cols].apply(lambda x: 100 - x.isna().mean() * 100).T
col_order = have_pct.sum().sort_values().index
have_pct = have_pct[col_order]

# pretty-print tables (rounded) and add heatmap
from IPython.display import display

print("Date presence (%) by order_status (rows = date columns):")
display(have_pct.round(2).style.background_gradient(cmap='Reds', axis=None).format("{:.2f}%"))

# date columns comparison: create -> approve -> carrier -> customer
ts_pairs = [
    ('order_purchase_timestamp', 'order_approved_at', 'purchase → approve'),
    ('order_approved_at', 'order_delivered_carrier_date', 'approve → carrier'),
    ('order_delivered_carrier_date', 'order_delivered_customer_date', 'carrier → customer'),
]

for earlier, later, label in ts_pairs:
    # both timestamps must be present to compare
    mask = orders[earlier].notna() & orders[later].notna() & (orders[later] < orders[earlier])
    count = mask.sum()
    pct = (count / mask.notna().sum() * 100) if mask.notna().sum() > 0 else 0
    print(f"{label}: {count} rows out of order ({pct:.2f}%)")


Date presence (%) by order_status (rows = date columns):


order_status,created,canceled,approved,invoiced,unavailable,processing,shipped,delivered
order_purchase_timestamp,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%
order_approved_at,0.00%,77.44%,100.00%,100.00%,100.00%,100.00%,100.00%,99.99%
order_delivered_carrier_date,0.00%,12.00%,0.00%,0.00%,0.00%,0.00%,100.00%,100.00%
order_delivered_customer_date,0.00%,0.96%,0.00%,0.00%,0.00%,0.00%,0.00%,99.99%
order_estimated_delivery_date,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%


purchase → approve: 0 rows out of order (0.00%)
approve → carrier: 1359 rows out of order (1.37%)
carrier → customer: 23 rows out of order (0.02%)


Based on the missingness of data, proposed stages & timestamps

| stage        | timestamp column              | rule                                                                               |
| ------------ | ----------------------------- | ---------------------------------------------------------------------------------- |
| created      | order_purchase_timestamp / order_estimated_delivery_date      | always present → creation event
| approved     | order_approved_at             | present only after payment approval                                                |
| shipped      | order_delivered_carrier_date  | present only when handed to carrier                                                |
| delivered    | order_delivered_customer_date | final delivery timestamp                                                           |
| stalled      | —                             | missing ship/delivery timestamps where status is *processing/unavailable/invoiced* |



### Table Keys
- **customers**
  - `customer_id` (PK)

- **orders**
  - `order_id` (PK)
  - `customer_id` (FK → customers)

- **order_items**
  - `order_id` (FK → orders)
  - `product_id` (FK → products)
  - `seller_id` (FK → sellers)

- **order_payments**
  - `order_id` (FK → orders)

- **order_reviews**
  - `order_id` (FK → orders)

- **sellers**
  - `seller_id` (PK)

- **products**
  - `product_id` (PK)


In [4]:
df = orders.copy()

def print_join_info(df, left, join_col, indicator_col):
    df = df.merge(left, on=join_col, how='left', indicator=indicator_col)
    join_counts = df[indicator_col].value_counts()
    print(f"Join info for {indicator_col}:")
    print(join_counts)
    return df
    
df = print_join_info(df, customers, 'customer_id', 'cust_joined')
df = print_join_info(df, orderitems, 'order_id', 'item_joined')
df = print_join_info(df, orderpayments, 'order_id', 'pay_joined')
df = print_join_info(df, orderreviews, 'order_id', 'review_joined')
df = print_join_info(df, sellers, 'seller_id', 'seller_joined')
df = print_join_info(df, products, 'product_id', 'prod_joined')

# no failed right joins, all good
# check if failed left joins arose from statuses

Join info for cust_joined:
cust_joined
both          99441
left_only         0
right_only        0
Name: count, dtype: int64
Join info for item_joined:
item_joined
both          112650
left_only        775
right_only         0
Name: count, dtype: int64
Join info for pay_joined:
pay_joined
both          118431
left_only          3
right_only         0
Name: count, dtype: int64
Join info for review_joined:
review_joined
both          118146
left_only        997
right_only         0
Name: count, dtype: int64
Join info for seller_joined:
seller_joined
both          118310
left_only        833
right_only         0
Name: count, dtype: int64
Join info for prod_joined:
prod_joined
both          118310
left_only        833
right_only         0
Name: count, dtype: int64


In [5]:
# Analyze order statues of failed left joins
failed_joins = ['cust_joined', 'item_joined', 'pay_joined', 'review_joined', 'seller_joined', 'prod_joined']
for col in failed_joins:
    failed_df = df[df[col] == 'left_only']
    status_counts = failed_df['order_status'].value_counts()
    print(f"Failed left joins for {col} by order_status:")
    print(status_counts)

# customers have no failed joins
# orderitems (and seller) failed joins mostly from unavailable and canceled orders
# payments failed joins from delivered
# reviews failed joins from various statuses

Failed left joins for cust_joined by order_status:
Series([], Name: count, dtype: int64)
Failed left joins for item_joined by order_status:
order_status
unavailable    645
canceled       180
created          5
invoiced         2
shipped          1
Name: count, dtype: int64
Failed left joins for pay_joined by order_status:
order_status
delivered    3
Name: count, dtype: int64
Failed left joins for review_joined by order_status:
order_status
delivered      861
shipped         88
canceled        20
unavailable     14
processing       6
invoiced         6
created          2
Name: count, dtype: int64
Failed left joins for seller_joined by order_status:
order_status
unavailable    645
canceled       180
created          5
invoiced         2
shipped          1
Name: count, dtype: int64
Failed left joins for prod_joined by order_status:
order_status
unavailable    645
canceled       180
created          5
invoiced         2
shipped          1
Name: count, dtype: int64


In [6]:
# fully joined OLAP dataset
df = orders \
    .merge(customers, on='customer_id', how='left') \
    .merge(orderitems, on='order_id', how='left') \
    .merge(orderpayments, on='order_id', how='left') \
    .merge(orderreviews, on='order_id', how='left') \
    .merge(customers, on='customer_id', how='left') \
    .merge(sellers, on='seller_id', how='left') \
    .merge(products, on='product_id', how='left')


In [7]:
# operations team datamart, calculate time between timestamps
dm_ops = df.copy()
dm_ops['time_to_approve'] = (df['order_approved_at'] - df['order_purchase_timestamp']).dt.total_seconds() / 3600.0
dm_ops['time_to_deliver'] = (df['order_delivered_customer_date'] - df['order_purchase_timestamp']).dt.total_seconds() / 3600.0
dm_ops['estimate_difference'] = (df[''] - df['order_purchase_timestamp']).dt.total_seconds() / 3600.0

# payments team datamart
dm_payments = df.copy()


KeyError: ''